In [23]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import lightgbm as lgb

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [16]:
df = pd.read_csv("../data/훈련데이터셋/3_훈련데이터셋_LAG_휴일포함.csv")
#df.info()

In [20]:
df.tail(5)

,날짜,전체코드,최저기온(°C),0.5m 지중온도(°C),평균 증기압(hPa),가조시간(hr),평균 상대습도(%),파워,lag_1d,lag_7d,휴일여부
233203,2024-01-24,1174011000,-11.2,2.3,1.7,10.1,52.0,115124.547,114960.859,108004.536,False
233204,2024-01-25,1174011000,-9.7,2.2,2.0,10.1,47.8,99907.500,115124.547,98758.864,False
233205,2024-01-26,1174011000,-6.9,2.1,3.4,10.1,62.5,94818.512,99907.500,NaN,False
233206,2024-01-27,1174011000,-4.6,2.0,4.0,10.2,70.0,86792.828,94818.512,76691.460,True
233207,2024-01-28,1174011000,-5.0,1.9,3.4,10.2,62.3,86389.477,86792.828,89640.934,True


In [21]:
feature_cols = [
    # 지역
    "전체코드",

    # 날씨
    
    "최저기온(°C)", "0.5m 지중온도(°C)", "평균 증기압(hPa)","가조시간(hr)","평균 상대습도(%)",

    # 기간

    "lag_1d", "lag_7d"

]
target_col = "파워"

In [22]:
new_df = df[["날짜"] + feature_cols + [target_col]].dropna().reset_index(drop=True)

In [24]:
# =========================
# 1) 시간순 Train/Valid/Test 분할
# =========================
new_df = new_df.sort_values("날짜").reset_index(drop=True)

n = len(new_df)
test_size = int(n * 0.20)
valid_size = int(n * 0.10)

train_end = n - (valid_size + test_size)
valid_end = n - test_size

train_df = new_df.iloc[:train_end]
valid_df = new_df.iloc[train_end:valid_end]
test_df  = new_df.iloc[valid_end:]

In [25]:
# =========================
# 2) X / y 분리
# =========================
drop_cols = [target_col, "날짜"]  # 날짜는 직접 입력 피처로 쓰지 않음(우린 파생변수로 이미 반영)
X_train = train_df.drop(columns=drop_cols)
y_train = train_df[target_col]

X_valid = valid_df.drop(columns=drop_cols)
y_valid = valid_df[target_col]

X_test = test_df.drop(columns=drop_cols)
y_test = test_df[target_col]

In [26]:

# =========================
# 3) 범주형 처리 (코드형 컬럼은 category로)
# =========================
cat_cols = []
for c in ["전체코드", "시군구코드", "법정동코드"]:
    if c in X_train.columns:
        cat_cols.append(c)

for c in cat_cols:
    X_train[c] = X_train[c].astype("category")
    X_valid[c] = X_valid[c].astype("category")
    X_test[c]  = X_test[c].astype("category")

In [27]:
# =========================
# 4) LightGBM Baseline 모델
# =========================
model = LGBMRegressor(
    objective="regression_l2",   # 🔥 반드시 변경

    n_estimators=20000,
    learning_rate=0.03,

    num_leaves=63,
    max_depth=10,
    min_child_samples=80,        # ↓ 완화
    min_split_gain=0.05,         # ↓ 완화

    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.8,

    reg_alpha=0.1,               # ↓
    reg_lambda=5.0,              # ↓

    random_state=42,
    n_jobs=-1
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="rmse",
    categorical_feature=cat_cols if len(cat_cols) > 0 else "auto",
    callbacks=[
        # early stopping (너무 오래 돌지 않게)
        # verbose는 200번마다 한번씩 로그
        __import__("lightgbm").early_stopping(stopping_rounds=200, verbose=True),
        __import__("lightgbm").log_evaluation(period=200),
    ]
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002318 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 106549, number of used features: 8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 141294.244140
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

LGBMRegressor(colsample_bytree=0.8, learning_rate=0.03, max_depth=10,
              min_child_samples=80, min_split_gain=0.05, n_estimators=20000,
              n_jobs=-1, num_leaves=63, objective='regression_l2',
              random_state=42, reg_alpha=0.1, reg_lambda=5.0, subsample=0.8,
              subsample_freq=1)

In [28]:
# =========================
# 5) 평가 (Valid / Test)
# =========================
def eval_print(name, y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    print(f"[{name}] RMSE: {rmse:,.3f}")
    print(f"[{name}] R2  : {r2:.6f}")
    return rmse, r2

pred_valid = model.predict(X_valid)
pred_test  = model.predict(X_test)

eval_print("VALID", y_valid, pred_valid)
eval_print("TEST",  y_test,  pred_test)

print("Best iteration:", model.best_iteration_)

[VALID] RMSE: 187,870.269
[VALID] R2  : 0.758467
[TEST] RMSE: 90,556.356
[TEST] R2  : 0.836646
Best iteration: 716
